## Visualization of Airport Data on Map.

The idea and the airport's dataset for this demo was found in the "Towards Data Science" article [How to Visualize Data on top of a Map in Python using the Geoviews library](https://towardsdatascience.com/how-to-visualize-data-on-top-of-a-map-in-python-using-the-geoviews-library-c4f444ca2929) by [Christos Zeglis](https://medium.com/@christoszeglis).


In [1]:
# !pip install lets-plot --quiet
import lets_plot
lets_plot.__version__

'4.5.1'

In [2]:
import numpy as np
import pandas as pd
from lets_plot import *
from lets_plot import tilesets

LetsPlot.setup_html()

### Load the "airports" dataset.

The "airports" dataset is already cleaned and only contains data on Greece and Turkey airports.

In [3]:
airports = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/airports.csv")
airports.head(3)

,IATA,city,latitude,longitude,passengers,country
0,ATH,Athens,37.9354,23.9437,24.13,GR
1,HER,Heraklion,35.3400,25.1753,8.00,GR
2,SKG,Thessaloniki,40.5230,22.9767,6.67,GR


### Fetch boundaries of Greece and Turkey.

Do it using the Lets-Plot **built-in geocoding**.

In [4]:
from lets_plot.geo_data import *

countries_gcoder = geocode_countries(['GR', 'TR'])
countries_gcoder.get_geocodes()

The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL).


,id,country,found name,centroid,position,limit
0,192307,GR,Greece,"[21.5772458904778, 39.6987419575453]","[20.0085201859474, 37.6486940681934, 26.635576...","[19.3734280765057, 34.8015573620796, 29.645195..."
1,174737,TR,Turkey,"[35.4993526199025, 38.95283639431]","[26.0636124014854, 35.807680785656, 44.8176634...","[25.6654492020607, 35.807680785656, 44.8176634..."


### Show Greece, Turkey and airports data on interactive map.

- Add an interactive base-map layer with custom initial location and zoom level.
- NASA's "CityLights 2012" map-tiles [© NASA Global Imagery Browse Services (GIBS)](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/gibs).
- Add polygons layer to fill the country boundaries with semi-transparent colors.
- Add points layer marking the airports location with the point size proportional to the airport's passengers volume.
- Customize tooltip on the points layer to show all the airport data.
- Use the 'scale_fill_manual()' function in order to fill polygons and points with blue and red colors.
- Setup the desired marker's size using the 'scale_size()' function.

In [5]:
(ggplot() 
 + geom_livemap(location=[26.65, 38.61],
                zoom=6,
                data_size_zoomin=-1,
                tiles=tilesets.NASA_CITYLIGHTS_2012)
 + geom_polygon(aes(fill='country'), 
                data=countries_gcoder.get_boundaries(), 
                size=0,
                alpha=.2)
 + geom_point(aes('longitude', 'latitude', fill='country', size='passengers'),
              data=airports,
              shape=21,
              alpha=.7,
              color='white',
              tooltips=layer_tooltips()
                        .format('passengers', '{.1f} m' )
                        .format('^x', '.2f').format('^y', '.2f')
                        .title('@city (@country)')
                        .line('@|@IATA')
                        .line('Passengers|@passengers') 
                        .line('Longitude|^x')
                        .line('Latitude|^y'))
 + scale_fill_manual(values=['#30a2da', '#fc4f30'])
 + scale_size(range=[10, 40], trans='sqrt')
 + theme(legend_position='none')
 + ggsize(1000, 620)
 
) 